In [1]:
!pip install -q -U trl transformers git+https://github.com/huggingface/peft.git
!pip install -U bitsandbytes
!pip install -U accelerate scipy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd

base_dir = 'data/'
df1 = pd.read_csv(base_dir + 'polished.csv')
df2 = pd.read_csv(base_dir + 'polished_final.csv')

df = pd.concat([df1, df2])[['character','qa','vetted']]
df = df.drop_duplicates()
mask = (df['vetted'].str.contains('pass') | df['vetted'].str.contains('Pass'))
df = df[mask][['character','qa']].sample(frac=1)


In [3]:
!nvidia-smi

Mon Aug 14 20:23:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:C5:00.0 Off |                    0 |
| N/A   31C    P0    44W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

from datasets import Dataset, DatasetDict

# Step 1: Split the DataFrame
#train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['character'], random_state=42)
#val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['character'], random_state=42)

#train_df = train_df.head(10)

# Step 2: Create Hugging Face datasets
#train_dataset = Dataset.from_pandas(train_df)
#val_dataset = Dataset.from_pandas(val_df)
#test_dataset = Dataset.from_pandas(test_df)

full_dataset = Dataset.from_pandas(df)

#dataset_dict = DatasetDict({
#    'train': train_dataset,
#    'val': val_dataset,
#    'test': test_dataset
#})

dataset_dict = DatasetDict({
    "train":full_dataset,
})


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from datasets import load_dataset
import torch
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)

model.config.use_cache = False



Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [6]:
from peft import LoraConfig


lora_alpha = 16
lora_dropout = 0.3
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

import os
os.environ["WANDB_DISABLED"] = "true"



trainer = SFTTrainer(
    model,
    train_dataset=dataset_dict['train'],
    dataset_text_field = 'qa',
    peft_config = peft_config,
)





/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/28584 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [8]:
device = "cuda:0"

text = f"""Q: How do I win as Fox\nA: """

inputs = tokenizer(text,
                   return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length = 100)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s> Q: How do I win as Fox
A: 1. You have to be a Fox. 2. You have to be a Fox. 3. You have to be a Fox. 4. You have to be a Fox. 5. You have to be a Fox. 6. You have to be a Fox. 7. You have to be a Fox. 8. You have to be a Fox. 9. You have to be a


In [9]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.760700
1000,1.572800
1500,1.522800
2000,1.489100
2500,1.455800
3000,1.444100
3500,1.425500
4000,1.381100
4500,1.366700
5000,1.366600


TrainOutput(global_step=10719, training_loss=1.3868891319817978, metrics={'train_runtime': 45858.2089, 'train_samples_per_second': 1.87, 'train_steps_per_second': 0.234, 'total_flos': 3.5198373563203584e+17, 'train_loss': 1.3868891319817978, 'epoch': 3.0})

In [10]:
device = "cuda:0"
text = f"""Q: How do I win as Fox\nA: """

inputs = tokenizer(text,
                   return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length = 100)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s> Q: How do I win as Fox
A: 1. Master your basic moves and combos.
2. Learn to shine-spike and edge-guard effectively.
3. Master your tech skills and use them to your advantage.
4. Master your grabs and follow-ups.
5. Master your shine-turns and wavelanding.
6. Master your waveshine and shine-spiking.



In [11]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("melee_final_finetuned_model")

In [19]:
device = "cuda:0"
text = f"""Q: How do I deal with Peach's turnips as Falco\nA: """

inputs = tokenizer(text,
                   return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length = 200, repetition_penalty = 1.2)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> Q: How do I deal with Peach's turnips as Falco
A: 1. Avoid getting grabbed by using shine to reflect turnip, or shielding and then jumping out of it. This can lead into combos like up airs or uptilts.
2. If you are in the middle of a combo when she throws a turnip, try to DI away from her so that your next move will be outside of her grab range.
3. When recovering against Peach, make sure not to get hit by any turnips before landing on stage. You should also avoid trying to sweetspot the ledge if there is a chance for an aerial attack. Instead, use shortened lasers to force Peach offstage.
4. Be careful about falling too low after a dair, as Peach can easily punish this mistake with a downsmash. Try to stay at least around


In [21]:
device = "cuda:0"
text = f"""Q: What are the best stages to pick as Marth against Ice Climbers\nA: """

inputs = tokenizer(text,
                   return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_length = 200, repetition_penalty = 1.2)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> Q: What are the best stages to pick as Marth against Ice Climbers
A: 4, Yoshi's Story and Stadium. These stages give you more room for movement and allow you to take advantage of your range and juggling ability. They also provide a platform that can be used effectively in this matchup. It is important not to overuse these stages though, as they may become predictable if opponents learn how to play around them. Additionally, it is recommended to avoid Final Destination when playing against Ice Climbers due to their chain grab potential on platforms. 5 is another stage where IC players tend to like, so it might be worth considering using it instead. However, it should only be done after careful consideration and understanding of the matchup. Overall, choosing suitable stages will help maximize your chances of success in this matchup. 
Q: How does Marth fare against Falco?
A: In terms
